# Genshin Impact Guide

I copied some longform wiki lore from https://genshin-impact.fandom.com/wiki/Timeline

It's not automated which is a shame because I couldn't find a simple way to get the raw content from the website. So I manually copied and cleaned up some of the text.

Available at assets/genshin_impact.txt

In [24]:
!pip install langchain langchain-text-splitters langchain-openai langchain-community openai tiktoken rapidocr-onnxruntime faiss-gpu python-dotenv

In [2]:
from platform import python_version
print(python_version())

3.10.12


In [3]:
import langchain_text_splitters
import langchain_community.document_loaders

In [4]:
text_loader = langchain_community.document_loaders.TextLoader("assets/genshin_impact.txt")

In [5]:
document = text_loader.load()

In [6]:
splitter = langchain_text_splitters.RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

In [7]:
chunks = splitter.split_documents(document)

In [8]:
chunks[:10]

[Document(metadata={'source': 'assets/genshin_impact.txt'}, page_content="The Old World\n\nAll events take place an unknown time in the past, but allegedly predate Morax's birth over 6,000 years ago."),
 Document(metadata={'source': 'assets/genshin_impact.txt'}, page_content="The Seven Sovereigns\nDragons and vishaps are the dominant species, led by the Seven Sovereigns and the Dragon King Nibelung.\nAll elemental beings come from the Light Realm, also known as the Vishap Realm.\nEach of the Seven Sovereigns holds one of the seven elemental Authorities, and they are the strongest dragon of their respective element. The world is divided among them, and each one's respective land is attuned to their elemental affinity."),
 Document(metadata={'source': 'assets/genshin_impact.txt'}, page_content='In Sumeru, Apep holds the Dendro elemental Authority. She rules the land while nurturing some of the most ancient and primordial beings, which are known simply as Dendro lifeforms.'),
 Document(me

In [9]:
%load_ext dotenv
%dotenv

In [10]:
import os
import langchain.chains
import langchain_openai
import langchain_core
import langchain_community.vectorstores

In [11]:
embeddings = langchain_openai.OpenAIEmbeddings(api_key=os.environ["OPENAI_API_KEY"])

In [12]:
in_mem_vecs = langchain_community.vectorstores.FAISS.from_documents(chunks, embeddings)

In [13]:
retriever = in_mem_vecs.as_retriever()

In [14]:
prompt = langchain_core.prompts.ChatPromptTemplate.from_template("""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
""")

In [15]:
model = langchain_openai.ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"], model="gpt-3.5-turbo")

In [16]:
output_parser = langchain_core.output_parsers.StrOutputParser()

In [17]:
#from langchain.chains.combine_documents import create_stuff_documents_chain
#documents_chain = langchain.chains.combine_documents.create_stuff_documents_chain(model, prompt)

In [18]:
#retrieval_chain = langchain.chains.create_retrieval_chain(retriever, documents_chain)

In [19]:
#retrieval_chain.invoke({"input": "Tell me Mona's backstory."})

In [20]:
retrieval_chain = {"context": retriever,  "question": langchain_core.runnables.RunnablePassthrough()} | prompt | model | output_parser

In [21]:
retrieval_chain.invoke("Tell me Mona's backstory.")

"Mona's backstory involves her being enslaved by the aristocracy in Mondstadt due to her great strength. She was made into a gladiator but the chains could not bind her. She was later found by Pierro of the Fatui Harbingers, who gave her a Cryo Delusion to put out her flames. This led to her transformation into Signora, the 8th of the Fatui Harbingers. Additionally, 1010 years ago, she fled to Mondstadt after encountering Ursa the Drake. Faruzan, who was trapped within desert ruins, escaped 100 years ago and was found with a Vision in a Sumeru forest. Cyno studied under the same master as Lisa at an unknown point in time. Faranak left for Fontaine for employment and Kaveh met Alhaitham, becoming friends."

In [22]:
retrieval_chain.invoke("What was Teyvat like before the traveller's appearance?")

"Before the Traveler's appearance in Teyvat, the land was plagued by the Beast of Thunder enslaving people with lightning. The Thundersoother, seeking revenge for his family's sacrifice, defeated the Beast and received a crown but lived in solitude. There were unknown regions in Teyvat where conflicts and battles took place. Invaders from beyond the firmament brought forbidden knowledge and diseases to Teyvat, causing havoc. The Seelie fought against these invaders but were unable to stop them. Khaenri'ah was destroyed by the Gods, witnessed by the Traveler's sibling before waking the Traveler. The one-eyed king Irmin grew weak, and the Alberich Clan took over as regents post-cataclysm. Overall, Teyvat was a land filled with conflicts, forbidden knowledge, and destruction before the Traveler's appearance."